In [1]:
#For future QA classification. Copy of classification.ipynb
import pandas as pd
import os

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import pickle

In [3]:
train1 = pd.read_csv('/home/andrei/Work/data/past_plans.csv') #1
train2 = pd.read_csv('/home/andrei/Work/data/records.csv') #0
df = train1.append(train2) 

In [4]:
train1

,text,class
0,What were my plans in January last year?,1
1,What were my plans for 14.06.2019?,1
2,What were my plans the day before yesterday?,1
3,What were my plans for last Wednesday?,1
4,What were my plans this time last year?,1
...,...,...
145,What did I have scheduled on Monday 26th April?,1
146,What did I have scheduled two weeks ago?,1
147,What did I have scheduled at the end of last m...,1
148,What did I have scheduled after 7 pm last Thur...,1


In [5]:
df["text_lower"]  = [text.lower() for text in df["text"]]

In [6]:
df

,text,class,text_lower
0,What were my plans in January last year?,1,what were my plans in january last year?
1,What were my plans for 14.06.2019?,1,what were my plans for 14.06.2019?
2,What were my plans the day before yesterday?,1,what were my plans the day before yesterday?
3,What were my plans for last Wednesday?,1,what were my plans for last wednesday?
4,What were my plans this time last year?,1,what were my plans this time last year?
...,...,...,...
142,Did I log anything before 12:00 yesterday?,0,did i log anything before 12:00 yesterday?
143,Did I log anything on July 12th 2020?,0,did i log anything on july 12th 2020?
144,Did I log anything on 08.07.2018?,0,did i log anything on 08.07.2018?
145,Did I log anything last Sunday evening?,0,did i log anything last sunday evening?


In [7]:
#vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
vectorizer = CountVectorizer(min_df= 1, ngram_range=(1, 1))
final_features = vectorizer.fit_transform(df['text_lower']).toarray()
final_features.shape
len(sorted(list(vectorizer.vocabulary_)))

165

In [8]:
X = df['text_lower']
Y = df['class']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [10]:
pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k='all')), #1200
                     ('clf', LogisticRegression(random_state=0))])

In [11]:
model = pipeline.fit(X_train, y_train) #build model
#print(os.getcwd())


In [12]:
saved_model = os.getcwd() + "/" + "model3.pkl"
with open(saved_model, 'wb') as file:
    pickle.dump(model, file)
#with open(saved_model, 'rb') as file:
#    model = pickle.load(file)
ytest = np.array(y_test)

In [13]:
print(classification_report(ytest, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      0.92      0.95        38
           1       0.92      0.97      0.95        37

    accuracy                           0.95        75
   macro avg       0.95      0.95      0.95        75
weighted avg       0.95      0.95      0.95        75



In [14]:
print(confusion_matrix(ytest, model.predict(X_test)))

[[35  3]
 [ 1 36]]


In [15]:
len(df)

297

In [17]:
test = ['What records did I make yesterday?']
#test = ['Did I do anything last September?']
model.predict(test)
#model.predict_proba(test) #time 1 loc 0

array([0])

In [217]:
test = [line.strip("\n") for line in open(os.getcwd()+"/"+"plans4_test") if line!='\n']
test

['Make raspberry jelly today.',
 'Show favourite film at film night.',
 'Dig over unused area of vegetable patch next.',
 'Tidy all parts of shed.',
 'Wear tallest elaborate wig today.',
 'Make carbonara sauce for pasta.',
 'Put a tall, evergreen shrub near fence.',
 'Pair a strong blue cheese with the wine.',
 'Buy a practical and well-made bicycle.',
 'Audition for all the best parts.',
 'Turn up radiator before getting out of bed.',
 'Prune plum tree after threat of frost has passed.',
 'Start planning party straight away.',
 'Visit new art exhibition Tuesday afternoon.',
 'Switch off radio when leaving room.',
 'Restoration scheduled to begin end of March.',
 'Next available appointment 23rd March.',
 'Show due to start filming 2nd May.',
 'Deadline for applications in two weeks.',
 'House viewing booked for Monday 5th April.',
 'Report due for publication in September.',
 "My sister's flight is at 8 pm.",
 'Grandmother due on stage 9.30 pm.',
 'New table due to be delivered this a

In [19]:
#test = ["when is the meeting" , "when is smth"]

#model.predict(test)
model.predict_proba(test) #time 1 loc 0

array([[0.8471124, 0.1528876]])

In [19]:
final_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [20]:
len(final_features)

400

In [21]:
final_features[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
final_features[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,